# Prerequisitos

Para ejecutar el script es necesario instalar la siguientes librerias Python:

* pip install pandas
* pip install re
* pip install numpy
* pip install urllib3

# Ejecutar Script

1. Se debe crear un directorio data en la misma ruta donde se encuentra el script.
2. Ejecutar el script __\# python3 foodPriceCorabastos.py__.
3. En directorio __data__ se encuentra el archivo .csv con los datos extraídos del sitio web.


# Código fuente

__Cargamos las librerias Phython__

In [1]:
import urllib3 # Libreria para descargar URL
import re # Libreria para exprecion regular
import numpy as np # Libreria para crear manipulacion de array
import pandas as pd # Libreria para crear manipulacion de DataFrame

## Definicion de funciones
********


**_Funcion:download_**

Funcion para descargar el sitio web

In [2]:
# download: funcion para descargar el sitio web
# parametros in:
# url: direccion del sitio web
# num_retries: numeros de reintentos de la solicitud en caso de error
# user_agent: informacion de la peticion por defecto el valor 'wswp' (Estandar web scraping con Python)
def download(url,user_agent='wswp',num_retries=2):
    print ('Downloading:', url)
    
    headers = {'User-agent': user_agent}
    
    http = urllib3.PoolManager()
    response = http.request('GET', url, retries=False,headers=headers)
    
    if response.status == 200:
        html = response.data
    else:
        print('Download error: cod_status => ', response.status)
        html = None
        if num_retries > 0:            
            if 500 <= response.status < 600:                
                # retry 5XX HTTP errors                
                return download(url, user_agent, num_retries-1)
    
    return html.decode('utf-8')

**_Funcion: extractData_**

Esta funcion extrae los datos del HTML, por medio de patrones utilizando expreciones regulares

In [3]:
# extractData: extrae los datos del HTML
# parametros in:
# html: HTML de la pagina web descargada
def extractData(html):
    colum_names = ['grupo','nombre','presentacion','cantidad','unidad','cal_extra','cal_primera','valor','fecha_publicacion']
    array_html = re.findall('<img src="../img/logosolo.gif" align="bottom" />(.*?)<br />.*?<div class="table-responsive">(.*?)</div><b>', html,flags=re.DOTALL)
    df_data = pd.DataFrame(columns=colum_names)
    for html_grupo in array_html:
        grupo = re.findall('(.*?) Actualizado el: (.{10})', html_grupo[0],flags=re.DOTALL)
        tbody_tr = re.findall('<tbody>(.*?)</tbody>', html_grupo[1],flags=re.DOTALL)
        array_data = re.findall('<tr.*?>.*?<td.*?>(.*?)</td>.*?<td.*?>(.*?)</td>.*?<td.*?>(.*?)</td>.*?<td.*?>(.*?)</td>.*?<td.*?>(.*?)</td>.*?<td.*?>(.*?)</td>.*?<td.*?>(.*?)</td>.*?</tr>', tbody_tr[0],flags=re.DOTALL)
        
        #creamos array con los datos
        col_grupo = np.full((len(array_data),1), grupo[0][0]) # crear columna grupo
        col_fpublicacion = np.full((len(array_data),1), grupo[0][1]) # crear columna fecha publicacion
        all_data = np.hstack((col_grupo,array_data,col_fpublicacion)) # crear dataset

        df_data = df_data.append(pd.DataFrame(all_data,columns=colum_names),ignore_index=True)
        #print(array_data)
        #print(all_data)
    return df_data

## Descargamos HTML del sitio web

In [4]:
url = 'https://www.corabastos.com.co/sitio/historicoApp2/reportes/prueba.php'
html = download(url)
# print("html => ", html)

Downloading: https://www.corabastos.com.co/sitio/historicoApp2/reportes/prueba.php


/home/datamining/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


## Extraer los datos del HTML y Creamos el DataSet

In [5]:
df_data = extractData(html)
df_data.head(10) # mostramos las primeras 10 filas del dataset


,grupo,nombre,presentacion,cantidad,unidad,cal_extra,cal_primera,valor,fecha_publicacion
0,HORTALIZAS,ACELGA,ATADO,10.00,KILO,"$ 10,000","$ 9,000","$ 1,000",2019-10-29
1,HORTALIZAS,AHUYAMA,KILO,1.00,KILO,"$ 1,300","$ 1,100","$ 1,300",2019-10-29
2,HORTALIZAS,AJO ROSADO,ATADO,5.00,KILO,"$ 45,000","$ 43,000","$ 9,000",2019-10-29
3,HORTALIZAS,ALCACHOFA,DOCENA,5.00,KILO,"$ 25,000","$ 23,000","$ 5,000",2019-10-29
4,HORTALIZAS,APIO,ATADO,10.00,KILO,"$ 12,000","$ 10,000","$ 1,200",2019-10-29
5,HORTALIZAS,ARVEJA VERDE,BULTO,50.00,KILO,"$ 190,000","$ 185,000","$ 3,800",2019-10-29
6,HORTALIZAS,BERENJENA,KILO,1.00,KILO,"$ 1,300","$ 1,100","$ 1,300",2019-10-29
7,HORTALIZAS,BROCOLI,DOCENA,12.00,KILO,"$ 30,000","$ 28,000","$ 2,500",2019-10-29
8,HORTALIZAS,CALABACIN,KILO,1.00,KILO,"$ 1,500","$ 1,300","$ 1,500",2019-10-29
9,HORTALIZAS,CALABAZA,KILO,1.00,KILO,"$ 1,500","$ 1,300","$ 1,500",2019-10-29


## Exportamos el DataSet a un archivo con formato CSV

In [6]:
#print(df_data.loc[0,'fecha_publicacion'])
df_data.to_csv('../data/datos_' + df_data.loc[0,'fecha_publicacion'] + '.csv',index=False)